In [13]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [22]:
df = pd.read_csv('네이버쇼핑검색결과-에어팟.csv', engine='python', encoding='utf8')
#추정 원인은 pandas에서 사용하는 CParserWrapper의 자체 버그이며, 해결방법
#Python 3.6에서 한글파일명이 포함된 CSV 읽을 경우 에러가 날 떄가 있습니다.

In [23]:
df.head(3)

,category_list,create_at,img_url,is_ad,name,nv_mid,price,review_count,spec_dict,star+score,url
0,"['디지털/가전', '음향가전', '블루투스셋', '블루투스이어폰/이어셋']",2016.12.,https://shopping-phinf.pstatic.net/main_107769...,False,애플 에어팟 1세대 (MMEF2KH/A),10776906666,214310,11932.0,"{'용도': '통화+음악(스테레오)', '채널': '스테레오', '무선방식': '블...",4.8,http://cr2.shopping.naver.com/adcr.nhn?x=laZEG...
1,"['디지털/가전', '음향가전', '블루투스셋', '블루투스이어폰/이어셋']",2019.04.,https://shopping-phinf.pstatic.net/main_186220...,False,애플 에어팟 2세대 유선충전 (MV7N2KH/A),18622086330,209000,151.0,"{'종류': '이어셋', '용도': '통화+음악(스테레오)', '유닛형태': '코드...",4.8,http://cr2.shopping.naver.com/adcr.nhn?x=m8WjU...
2,"['디지털/가전', '음향가전', '블루투스셋', '블루투스이어폰/이어셋']",2019.04.,https://shopping-phinf.pstatic.net/main_184837...,False,해외애플 에어팟 2세대 무선충전,18483728924,235320,108.0,"{'종류': '이어셋', '용도': '통화+음악감상겸용', '무선방식': '블루투스'}",4.9,http://cr2.shopping.naver.com/adcr.nhn?x=xGh%2...


In [71]:
for detail_url in df.url:
    print(detail_url)
    break

http://cr2.shopping.naver.com/adcr.nhn?x=laZEGW%2BtWgOSR6SSnbG4c%2F%2F%2F%2Fw%3D%3DsiEky1DWIRp9RPCm8ky0MMhK519%2FYQpLyuzJwjI0lwM%2FwMHFVfyRR41kkqop4BBe7b8mPFshxxZjUZilPFxJ49LOzWylEKLxqT9tnjLp1ZU9v5vPmSTHMhkGjhp5a56X4H%2Ba899jq4vPrE%2FGLlLG4%2FEZVgIO42F1ZaXskE4po%2F46fokx%2Bl4FoGy1QI9MhYsm97ezzdZ5SmNRKo6EhCioDiHib%2FRhqgYr9wnO%2FynfKSkAY3VoGsDE1XT1S6%2BOsYQTY%2BHjcviF3tCdK3HNpy49BDe8UEBRSWcEZo8%2BvK9eep8YAlHlfLvZE7oHvTvHgz2fijoOGrgwaFoZ%2B%2FQd5RATpAH0uxbhGCKlU8oD%2FYg3hwAniaaJza9DyXc4%2B890sFMHn3BTNc1ou8z07L6rEizbO%2BexhKqqqXKolPp7yRuyM%2B8sWG5xH%2BMDbHiNNiHNZeSlzRCmbo9tv%2Flwtg6f%2BJDJpivGNqpKCAAb85PIlxyhmhd5rPVzI8i4BESS239iSmFOWt5SQ7h9NlJklPiIKyYWpqZg7V7k688qcc3DL6Tr6EA5flfkORtuLbnKw6P2b3CvjwS2WtbSXzYjLNwsTUnaO6iSM7mfoYBKC6ortqtTwHymWQUZa3Xmw4kclqHoMSwVEf6V6dWNLrR48RgpR6lUzeAz8in5VLtp6p%2F8exCeAYHhOAF5ksHaPhHo2JPalOVV7Ax3vTX3vA2N5iwtHqwkD6A%3D%3D


In [72]:
detail_url = df.url[1]

## 크롤링 시작

In [73]:
res = requests.get(detail_url)
res

<Response [200]>

In [74]:
html = res.text
soup = BeautifulSoup(html, 'html.parser')

In [75]:
tag_list = soup.select('.sub_list li a')
len(tag_list)

9

In [76]:
tag_list[0]

<a class="_topics" data-review-seq="" data-topic-code="" href="#">
<span>주제전체</span>
</a>

In [77]:
tag_list[1]

<a class="_topics" data-review-seq="" data-topic-code="price" href="#"><span>가격</span></a>

In [78]:
topic_dict = {}

for tag in tag_list:
    topic_code = tag['data-topic-code']
    topic_name = tag.text.strip()
    #print(topic_code, topic_name)
    topic_dict[topic_name] = topic_code

topic_dict

{'주제전체': '',
 '가격': 'price',
 '음질': 'sound',
 '만족도': 'total',
 '충전': 'charging',
 '디자인': 'design',
 '기능': 'function',
 '품질': 'quality',
 '사용성': 'usability'}

## Review_rist

In [79]:
review_list_url = 'https://search.shopping.naver.com/detail/review_list.nhn'

params = 'nvMid=10776906666&page=1&reviewSort=accuracy&reviewType=all&topicCode=price&ligh=true'
params.split('&')

['nvMid=10776906666',
 'page=1',
 'reviewSort=accuracy',
 'reviewType=all',
 'topicCode=price',
 'ligh=true']

In [80]:
for row in params.split('&'):
    print('"{}" : "{}"'.format(*row.split('=')))

"nvMid" : "10776906666"
"page" : "1"
"reviewSort" : "accuracy"
"reviewType" : "all"
"topicCode" : "price"
"ligh" : "true"


In [81]:
review_list_url = 'https://search.shopping.naver.com/detail/review_list.nhn'
params = {
    "nvMid" : "10776906666",
    "page" : 1,
    "reviewSort" : "accuracy", # 랭킹순(accuracy), 최신순(registration)
    "reviewType" : "all", # 전체(all). 포토리뷰(photo)
    # "topicCode" : topic_dict['가격'],    "price", 넣얻 되고 안넣어도 되고
    "score" : 1, # 1-5 지정않아면 전체
    "ligh" : "true",
}
res = requests.post(review_list_url, params=params) # get에서 post로 변경
res

<Response [200]>

In [82]:
html = res.text
soup = BeautifulSoup(html, 'html.parser')

In [84]:
tag_list = soup.select('#_review_list > li')
len(tag_list)

20

In [86]:
tag = tag_list[0]


In [88]:
star_score = int(tag.select_one('.curr_avg').text)
star_score

1

In [90]:
path = tag.select_one('.info .path').text # 쇼핑몰
author = tag.select_one('.info .name').text # 구매자
path, author

('엘롯데', 'ehdr****님')

In [93]:
date = tag.select_one('.date').text # 작성날짜

In [95]:
subject = tag.select_one('.subjcet, .subject').text.strip() #css 선택자에서 ,(쉼표)는 or의 용도로 오타가 있는경우를 대비해서 입력해줌
subject

'개봉하자말자 이어폰 양쪽 둘다 들리더니 갑자기 오른쪽이 안들림 또 되다가 안되다가 알고보니 페어링이 문제였던거임 내폰이 문제인가 싶어 다른 아이폰에 연결해봄 똑같음 불량인줄 모르고'

In [97]:
content = tag.select_one('.atc').text.strip()
content

'개봉하자말자 이어폰 양쪽 둘다 들리더니 갑자기 오른쪽이 안들림 또 되다가 안되다가 알고보니 페어링이 문제였던거임 내폰이 문제인가 싶어 다른 아이폰에 연결해봄 똑같음 불량인줄 모르고 보냈겠지만 여기서 구매를 했으면 반품 교환을 해줘야 된다고 생각함 그런데 무슨 지침 무슨 지침 이러면서 안된다함 저번에도 엘롯데에서 가디건 사는데 재고 있다하면서 배송이 2주나 늦어짐 다시는 롯데 이용 안할거임 나몰라라하는데 업체측이랑 고객센터 말이 둘다 다름 롯데?서비스 쪽에선 한계가 있는듯^^다시는 이용안함 에어팟 불량옴~'

In [102]:
# shift + m 으로 합칠 수 있음

commnet_list = []
for tag in tag_list:
    star_score = int(tag.select_one('.curr_avg').text)
    path = tag.select_one('.info .path').text # 쇼핑몰
    author = tag.select_one('.info .name').text # 구매자
    date = tag.select_one('.date').text # 작성날짜
    subject = tag.select_one('.subjcet, .subject').text.strip() #css 선택자에서 ,(쉼표)는 or의 용도로 오타가 있는경우를 대비해서 입력해줌
    content = tag.select_one('.atc').text.strip()
    comment = {
        'star_score' : star_score,
        'path' : path,
        'author' : author,
        'date' : date,
        'subject' : subject,
        'content' : content,
    }
    commnet_list.append(comment)

commnet_list[0]
    
    


{'star_score': 1,
 'path': '엘롯데',
 'author': 'ehdr****님',
 'date': '2019.02.20.',
 'subject': '개봉하자말자 이어폰 양쪽 둘다 들리더니 갑자기 오른쪽이 안들림 또 되다가 안되다가 알고보니 페어링이 문제였던거임 내폰이 문제인가 싶어 다른 아이폰에 연결해봄 똑같음 불량인줄 모르고',
 'content': '개봉하자말자 이어폰 양쪽 둘다 들리더니 갑자기 오른쪽이 안들림 또 되다가 안되다가 알고보니 페어링이 문제였던거임 내폰이 문제인가 싶어 다른 아이폰에 연결해봄 똑같음 불량인줄 모르고 보냈겠지만 여기서 구매를 했으면 반품 교환을 해줘야 된다고 생각함 그런데 무슨 지침 무슨 지침 이러면서 안된다함 저번에도 엘롯데에서 가디건 사는데 재고 있다하면서 배송이 2주나 늦어짐 다시는 롯데 이용 안할거임 나몰라라하는데 업체측이랑 고객센터 말이 둘다 다름 롯데?서비스 쪽에선 한계가 있는듯^^다시는 이용안함 에어팟 불량옴~'}

In [105]:
df_comment = pd.DataFrame(commnet_list)
print(df_comment.shape)
df_comment.head(3)

(20, 6)


,author,content,date,path,star_score,subject
0,ehdr****님,개봉하자말자 이어폰 양쪽 둘다 들리더니 갑자기 오른쪽이 안들림 또 되다가 안되다가 ...,2019.02.20.,엘롯데,1,개봉하자말자 이어폰 양쪽 둘다 들리더니 갑자기 오른쪽이 안들림 또 되다가 안되다가 ...
1,mous****님,남자친구 선물로 구매했는데 열어보더니 중고 아니냐고 하더군여 저도 봤는데 원래 마감...,2019.03.26.,M mall,1,남자친구 선물로 구매했는데 열어보더니 중고 아니냐고 하더군여 저도 봤는데 원래 마감...
2,leem****님,동생 선물주려고 구입했는데 다른 에어팟 기기처럼 바로 연결이 안돼서 밤새 연결 해보...,2019.05.10.,에스투피,1,동생 선물주려고 구입했는데 다른 에어팟 기기처럼 바로 연결이 안돼서 밤새 연결 해보...


In [109]:
def get_review_list(nvMid, page=1, reviewSort='accuracy', reviewType = 'all', topicCode=None
                   ,score = None):

    review_list_url = 'https://search.shopping.naver.com/detail/review_list.nhn'
    params = {
        "nvMid" : nvMid,
        "page" : page,
        "reviewSort" : reviewSort, # 랭킹순(accuracy), 최신순(registration)
        "reviewType" : reviewType, # 전체(all). 포토리뷰(photo)
        "ligh" : "true",
    }
    
    if topicCode:
        params['topicCode'] = topicCode
    if score:
        params['score'] = score
    
    res = requests.post(review_list_url, params=params) # get에서 post로 변경
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    tag_list = soup.select('#_review_list > li')
    
    commnet_list = []
    for tag in tag_list:
        star_score = int(tag.select_one('.curr_avg').text)
        path = tag.select_one('.info .path').text # 쇼핑몰
        author = tag.select_one('.info .name').text # 구매자
        date = tag.select_one('.date').text # 작성날짜
        subject = tag.select_one('.subjcet, .subject').text.strip() #css 선택자에서 ,(쉼표)는 or의 용도로 오타가 있는경우를 대비해서 입력해줌
        content = tag.select_one('.atc').text.strip()
        comment = {
            'star_score' : star_score,
            'path' : path,
            'author' : author,
            'date' : date,
            'subject' : subject,
            'content' : content,
        }
        commnet_list.append(comment)

    return commnet_list 

In [110]:
get_review_list

<function __main__.get_review_list(nvMid, page=1, reviewSort='accuracy', reviewType='all', topicCode=None, score=None)>

In [113]:
df.iloc[2]

category_list           ['디지털/가전', '음향가전', '블루투스셋', '블루투스이어폰/이어셋']
create_at                                                 2019.04.
img_url          https://shopping-phinf.pstatic.net/main_184837...
is_ad                                                        False
name                                             해외애플 에어팟 2세대 무선충전
nv_mid                                                 18483728924
price                                                       235320
review_count                                                   108
spec_dict         {'종류': '이어셋', '용도': '통화+음악감상겸용', '무선방식': '블루투스'}
star+score                                                     4.9
url              http://cr2.shopping.naver.com/adcr.nhn?x=xGh%2...
Name: 2, dtype: object

In [117]:
review_list = get_review_list('18483728924', page=1)

In [119]:
import time

review_list = []
for page in range(1,4):
    review_list.extend(get_review_list('18483728924', page=1))
    time.sleep(0.1)
len(review_list)

60

In [120]:
df_review_list = pd.DataFrame(review_list)
print(df_review_list.shape)
df_review_list.head(3)

(60, 6)


,author,content,date,path,star_score,subject
0,hell****님,5월 7일 주문해서 5월 10일에 물건을 받았습니다. 배송 진짜 빠르네요. 해외 배...,2019.05.11.,민트홍콩,5,5월 7일 주문해서 5월 10일에 물건을 받았습니다. 배송 진짜 빠르네요. 해외 배...
1,dms*****님,6일 밤에 주문했는데 11일 오후에 왔어요 중국 노동절에 해외배송인데도 개인적으로 ...,2019.05.11.,위메프,5,6일 밤에 주문했는데 11일 오후에 왔어요 중국 노동절에 해외배송인데도 개인적으로 ...
2,ovo0****님,구입하고 4일만에 배송왔어요. 정품 등록도 구입한날로하니 됩니다. 왜 지금까지 망설...,2019.05.10.,민트홍콩,5,구입하고 4일만에 배송왔어요. 정품 등록도 구입한날로하니 됩니다. 왜 지금까지 망설...


In [126]:
# shift + tab으로 설명 볼 수 있음.

df_review_list.sort_values('date', ascending=False)

,author,content,date,path,star_score,subject
27,minv****님,배송도너무빠르고 상품도좋아여,2019.05.12.,민트홍콩,5,배송도너무빠르고 상품도좋아여
53,fufu****님,이건 머 말이필요없쥬배송 엄청 빠르고 그냥 아주..짱입니다요.,2019.05.12.,민트홍콩,5,이건 머 말이필요없쥬배송 엄청 빠르고 그냥 아주..짱입니다요.
13,fufu****님,이건 머 말이필요없쥬배송 엄청 빠르고 그냥 아주..짱입니다요.,2019.05.12.,민트홍콩,5,이건 머 말이필요없쥬배송 엄청 빠르고 그냥 아주..짱입니다요.
47,minv****님,배송도너무빠르고 상품도좋아여,2019.05.12.,민트홍콩,5,배송도너무빠르고 상품도좋아여
7,minv****님,배송도너무빠르고 상품도좋아여,2019.05.12.,민트홍콩,5,배송도너무빠르고 상품도좋아여
33,fufu****님,이건 머 말이필요없쥬배송 엄청 빠르고 그냥 아주..짱입니다요.,2019.05.12.,민트홍콩,5,이건 머 말이필요없쥬배송 엄청 빠르고 그냥 아주..짱입니다요.
0,hell****님,5월 7일 주문해서 5월 10일에 물건을 받았습니다. 배송 진짜 빠르네요. 해외 배...,2019.05.11.,민트홍콩,5,5월 7일 주문해서 5월 10일에 물건을 받았습니다. 배송 진짜 빠르네요. 해외 배...
40,hell****님,5월 7일 주문해서 5월 10일에 물건을 받았습니다. 배송 진짜 빠르네요. 해외 배...,2019.05.11.,민트홍콩,5,5월 7일 주문해서 5월 10일에 물건을 받았습니다. 배송 진짜 빠르네요. 해외 배...
1,dms*****님,6일 밤에 주문했는데 11일 오후에 왔어요 중국 노동절에 해외배송인데도 개인적으로 ...,2019.05.11.,위메프,5,6일 밤에 주문했는데 11일 오후에 왔어요 중국 노동절에 해외배송인데도 개인적으로 ...
41,dms*****님,6일 밤에 주문했는데 11일 오후에 왔어요 중국 노동절에 해외배송인데도 개인적으로 ...,2019.05.11.,위메프,5,6일 밤에 주문했는데 11일 오후에 왔어요 중국 노동절에 해외배송인데도 개인적으로 ...


In [127]:
df_review_list.to_excel('리뷰리스트.xlsx')